In [14]:
from pathlib import Path
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Project root
PROJECT_ROOT = Path().resolve().parent
DATA_PATH = PROJECT_ROOT / 'data/superstoreSales.csv'
NOTEBOOKS_PATH = PROJECT_ROOT / 'notebooks'
os.makedirs(NOTEBOOKS_PATH, exist_ok=True)

# Load dataset
try:
    df = pd.read_csv(DATA_PATH, encoding='cp1252') #cp1252 or latin1
    print("Dataset loaded successfully with cp1252 encoding!")
except FileNotFoundError:
    print(f"Error: '{DATA_PATH}' not found.")
    print("Ensure 'superstoreSales.csv' is in 'data/' folder.")
    exit()
except Exception as e:
    print(f"Error loading dataset: {e}")
    exit()

# Check text fields for encoding issues
print("\n=== Sample Text Fields ===")
print(df[['Product Name', 'Customer Name', 'Product Category']].head(10))

# Basic inspection
print("\n=== Shape ===")
print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")

print("\n=== Columns ===")
print(df.columns.tolist())

print("\n=== Data Types ===")
print(df.dtypes)

print("\n=== Summary Statistics ===")
print(df[['Unit Price', 'Order Quantity', 'Shipping Cost', 'Discount', 'Profit', 'Product Base Margin']].describe())

print("\n=== Missing Values ===")
print(df.isnull().sum())

# Convert Order Date to datetime
df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')
if df['Order Date'].isnull().sum() > 0:
    print(f"Warning: {df['Order Date'].isnull().sum()} invalid Order Date entries.")

# Extract seasonality features
df['Month'] = df['Order Date'].dt.month
df['Quarter'] = df['Order Date'].dt.quarter
df['IsHoliday'] = df['Month'].isin([11, 12]).astype(int)  # Nov-Dec holidays

# Visualizations
sns.set(style="whitegrid")

# Histogram: Unit Price
plt.figure(figsize=(8, 5))
sns.histplot(df['Unit Price'], bins=30, kde=True)
plt.title('Unit Price Distribution')
plt.xlabel('Unit Price')
plt.ylabel('Frequency')
plt.savefig(NOTEBOOKS_PATH / 'unit_price_histogram.png')
plt.close()

# Histogram: Order Quantity
plt.figure(figsize=(8, 5))
sns.histplot(df['Order Quantity'], bins=30, kde=True)
plt.title('Order Quantity Distribution')
plt.xlabel('Order Quantity')
plt.ylabel('Frequency')
plt.savefig(NOTEBOOKS_PATH / 'order_quantity_histogram.png')
plt.close()

# Pairplot: Numerical features
sns.pairplot(df[['Unit Price', 'Order Quantity', 'Shipping Cost', 'Discount', 'IsHoliday']], hue='IsHoliday')
plt.savefig(NOTEBOOKS_PATH / 'pairplot.png')
plt.close()

# Boxplot: Unit Price by Product Category
plt.figure(figsize=(10, 5))
sns.boxplot(x='Product Category', y='Unit Price', data=df)
plt.title('Unit Price by Product Category')
plt.xticks(rotation=45)
plt.savefig(NOTEBOOKS_PATH / 'price_by_category.png')
plt.close()

# Time-series: Average Unit Price by Month
plt.figure(figsize=(10, 5))
df.groupby('Month')['Unit Price'].mean().plot(marker='o')
plt.title('Average Unit Price by Month')
plt.xlabel('Month')
plt.ylabel('Unit Price')
plt.grid(True)
plt.savefig(NOTEBOOKS_PATH / 'price_by_month.png')
plt.close()

# Correlation matrix
plt.figure(figsize=(8, 6))
corr = df[['Unit Price', 'Order Quantity', 'Shipping Cost', 'Discount', 'Product Base Margin']].corr()
sns.heatmap(corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix')
plt.savefig(NOTEBOOKS_PATH / 'correlation_matrix.png')
plt.close()

# Save dataset as UTF-8 (optional, for portability)
df.to_csv(PROJECT_ROOT / 'data/superstoreSales_utf8.csv', encoding='utf-8', index=False)
print("\nDataset saved as UTF-8: 'data/superstoreSales_utf8.csv'")

print("\nEDA completed! Visualizations saved in 'notebooks/' folder.")
print("Key features for modeling: Unit Price (target), Order Quantity, Shipping Cost, Month, IsHoliday, Product Category, Discount")
print("Preprocessing needs:")
print("- Handle missing values in Product Base Margin (~63 rows)")
print("- Encode categorical features: Product Category, Ship Mode, Order Priority")
print("- Check for outliers in Unit Price, Order Quantity")
print("- Verify text fields for encoding issues (e.g., Product Name)")

Dataset loaded successfully with cp1252 encoding!

=== Sample Text Fields ===
                                        Product Name       Customer Name  \
0   Eldon Base for stackable storage shelf, platinum  Muhammed MacIntyre   
1  1.7 Cubic Foot Compact "Cube" Office Refrigera...        Barry French   
2   Cardinal Slant-D¨ Ring Binder, Heavy Gauge Vinyl        Barry French   
3                                               R380       Clay Rozendal   
4                           Holmes HEPA Air Purifier      Carlos Soltero   
5  G.E. Longer-Life Indoor Recessed Floodlight Bulbs      Carlos Soltero   
6  Angle-D Binders with Locking Rings, Label Holders        Carl Jackson   
7            SAFCO Mobile Desk Side File, Wire Frame        Carl Jackson   
8              SAFCO Commercial Wire Shelving, Black      Monica Federle   
9                                          Xerox 198     Dorothy Badders   

  Product Category  
0  Office Supplies  
1  Office Supplies  
2  Office Supplies  
3